# Judy_Bot_T3 -- virtually 6th place NetHack agent comparing to [NetHack Challenge at NeurIPS 2021](https://www.aicrowd.com/challenges/neurips-2021-the-nethack-challenge) results

Version: 1.1.9 - AIF Exam Update

## Introduction
Released in 1987, NetHack is still one of the hardest video games ever published, with the victory that is achieved for the first time by the most determined players only after years of experience. Over time NetHack has attracted the attention of researchers and Artificial Intelligence enthusiasts, embodying a property-rich environment relevant to bring scientific progress to the state-of-the-art in Artificial Intelligence.

Within the game, the player will have to face fifty procedurally generated levels plus five extra floors, characterized by an even more extreme difficulty then the previous ones. The numerous monsters, the hidden traps and the constant need to feed the protagonist will be just some of the
elements that can lead the player to the “game over”, causing the entire game to start over.

Starting from the analysis of the thesis project Judy_Bot_T3, which aimed to create an open-source bot in Python for the videogame NetHack, the document will deal with the main characteristics of the developed agent and its evolutions in terms of what has been learned during the Artificial Intelligence Fundamentals course at University of Pisa.

As said, the agent Judy_Bot_T3 was developed as a thesis project following the bachelor's degree obtained on 02/12/2022.

Starting from version 1.1.7, first public version released in correspondence with the bachelor's degree in Computer Science, the agent was able to obtain an average score of 744 and a median score of 645. Scores acquired by playing the game exploiting the "NetHackChallenge-v0" task through the [NetHack Learning Environment](https://github.com/facebookresearch/nle) framework.

Starting from version 1.1.9 (AIF Exam Update) the agent is now able to obtain an average score of 1046.96 and a median score of 817.

The produced code is completely open-source and has been designed with the aim of complying with the principles of configurability, modularity and extensibility. It is in fact possible to extend the agent through the implementation of modules dedicated to the planning and execution of new tasks, which will become completely compatible with the main system.
<br>
<br>

## Related Works
Since its first release, the Judy_Bot_T3 project has been based on several previous studies, such as the implementations of the previous historical bots TAEB, BotHack and AutoAscend, as well as on two main frameworks: NetHack Learning Environment (NLE) and MiniHack.

NLE is a framework that aims to provide an interface to NetHack’s terminal software. To succeed in this objective, NLE formalises the game commands (input) in Python methods with appropriate configuration attributes, making the game output usable by the higher-level implementation through special observation data structures in accordance with the canons of the Gym interface on which it is founded.

MiniHack, on the other hand, is a framework geared towards the meticulous study of specific features of the otherwise vast NetHack environment. Thanks to those defined as “tasks” within the framework’s documentation, the developed agent can deal with restricted environments characterised by specific game elements in order to study a specific behaviour through limitations in the space of actions and observations available to the agent.

The same scientific personalities behind the development of these frameworks are also the creators of the NetHack NeurIPS Challenge, a competition that received a fair amount of attention during its 2021 edition, with the participation of more than four hundred and eighty developers and the submission of over forty implementations. 

<br>
<div>
<center><img src="img/ChallengeResults.jpg" width="500"/></center>
</div>
<br>

Thanks to the public results of the challenge, it was possible to compare Judy_Bot_T3’s results with those of the other candidates, following the (ordered for relevance) evaluation metric: ascensions number - median of scores - average score. These statistics played a key role in establishing the degree of success of the agent, which with an average score of 744 and a median score of 645 surpassed, in its first version (1.1.7), the eighth competitor: `JustPaulsAI`.

<br>
<div>
<center><img src="img/117Performance.jpg" width="500"/></center>
</div>
<br>

With the aim of its presentation in the Artificial Intelligence Foundamentals exam, Judy_Bot_T3 underwent numerous refinements both from code cleanliness and readability point of view (with the relocation of the component modules in separate files, the inclusion of comments describing each method and the writing of the README.md file, now merged with this notebook report) and from a functional point of view (with the refinement of existing modules according to the notions of the course and new elements learnt through further study of the game). The capabilities of the agent after the updates as well as the technical details of its implementation will be analyzed in the rest of the report, after which a further comparison with the statistics of the NetHack NeurIPS Challenge 2021 will be made for a final assessment.
<br>
<br>


## Methodologies
The ability to move efficiently within the NetHack map represents one of the most important implementations for an agent developed in this environment. For this purpose, Judy_Bot_T3 offers the "DungeonWalker" class, aimed at encapsulating functions useful for exploring the game world, including also all the implementations necessary for path finding. In order to elaborate its navigation plans, Judy_Bot_T3 makes use of the `A* Algorithm`, widely used at the state of the art. 

However, an important observation concerns the heuristics associated with the algorithm, a determining characteristic to ensure maximum performance in the case study environment. NetHack envisages that the playing character and other entities within the game can perform movements in the map grid in eight directions. Because of this characteristic of the environment, it proved essential to identify an appropriate admissible heuristic, with the design choice therefore falling on the `Octile Distance Heuristic` after a careful research phase.



In [ ]:
# heuristic for the distance between two points: A(ax, ay) and B(bx, by)
# environment with 8 directions of movement
def h_octile_distance (self, ay, ax, by, bx):
    x_d = abs(ax - bx)
    y_d = abs(ay - by)
    return (1.414 * min(x_d , y_d)) + abs(x_d - y_d)

The `Octile Distance Heuristic` evaluates a diagonal move as more costly than a move in the four directions, while giving it less weight than two moves in the same one, representing an admissible heuristic appropriate for problems such as NetHack. As an example, the following is a comparison of the identified “octile distance” heuristic with the famous “Manhattan distance” heuristic, known to be functional for searching in four-way environments.

As a numerical example, given the existence of a point A (0,0) and a point B (4,4), and considering the cost of a move in the four directions equal to 1, the `Manhattan Distance Heuristic` would predict a cost of moving from A to B equal to 8. Starting from the same premises, but looking at the case of the `Octile Distance Heuristic`, which predicts √2 as the cost of a diagonal movement action, the evaluation heuristic would yield a result equal to 5.66.

<br>
<div>
<center><img src="img/Heuristics.jpg" width="500"/></center>
</div>
<br>

It is therefore clear that the `Manhattan Distance Heuristic` applied to the environment under consideration would not lead to an approximation less than or equal to the true distance between the two points, resulting in a not admissible heuristic and consequently damaging the properties of the `A* Algorithm`.

Following the modifications made to the code in preparation for the Artificial Intelligence Foundamentals exam, the agent demonstrated significant improvements in the tasks defined by the “Elbereth”, “Eat” and “Fight” modules, in addition to the various slight enhancements of several minor sections of the code. Thanks to these changes, derived mainly from the in-depth study (facilitated by the MiniHack framework) of some specific game mechanics, the agent is now able to distinguish between healthy and unhealthy foods with an almost certain success rate, discarding suspect foods and thus avoiding potential risks. The agent is also now able to consistently protect itself through “Elbereth” engraving, taking advantage of the game mechanics whereby this action prevents most enemies from harming it. Thanks to a more refined logic, the bot will now avoid engraving when faced with
situations where it would not benefit from it.

Prayer is another powerful tool in the agent’s hands, and thanks to the latest changes, the agent can now make more optimized use of it.

Finally, some updates to the combat module now allow Judy_Bot_T3 to more successfully deal with specific enemies deserving of dedicated strategies, such as “floating eyes” and “werefoos”.
<br>
<br>


## Agent Assessment
The empirical evaluation of the agent was strongly guided by the comparison with the parameters and results of the NetHack NeurIPS Challenge 2021. Specifically, the code was run on a sample of 1000 games, all generated according to the parameters of the challenge and through
the use of “NetHackChallenge-v0” environment, available on the latest version of NLE (0.8.1). This environment comes as close as possible to playing the real game of NetHack with a random character to start, and a full keyboard of actions to take.

Collecting the outcomes of the games, the agent was then evaluated according to the three metrics: number of ascensions (wins) achieved, median and mean score between games. Having thus calculated the same metrics used in the challenge for the evaluation of the competitors, it was then possible to assess the quality of Judy_Bot_T3 in comparison to the latters. As the agent never achieved an ascension, as well as the other bots competing in the 2021
edition of the challenge, the pivotal parameter for the evaluation was the median of the scores between the games.

Thanks to the changes in the last few updates, the evaluation phase of Judy_Bot_T3 ended with the encouraging results of 1046.96 as the mean score and 817 as the median score, statistics that virtually rank the agent with a solid sixth position on the NetHack NeurIPS Challenge 2021 ranking list.

<br>
<div>
<center><img src="img/119Performance.jpg" width="500"/></center>
</div>
<br>


## Conclusions
The results achieved by Judy_Bot_T3 are certainly encouraging, as is the relative speed of improvement compared to previous thesis project results (around fifty to sixty hours of work). The collected data continue to demonstrate the very high potential of NetHack Learning Environment
and MiniHack frameworks in driving scientific research around the NetHack game.

It is clear, however that the journey towards the cration of a bot capable of “solving” the problem of the NetHack game is still incredibly long. Despite the challenge winners themselves have a huge difference in scores from what achieved by Judy_Bot_T3 (the winner, AutoAscend [5], boasts an unbeaten record of 5336.5 as a median score), the very same statistics of the podium agents are largely insufficient to consider the NetHack case of study as scientifically saturated.
<br>
<br>

## Team Contributions
Being the result of the evolution of the previous Thesis project, Judy_Bot_T3 has not seen the participation of a team for its realization and is presented for examination as an individual project.
<br>
<br>

## GitHub Metrics
As an individual work, the distribution of GitHub commits per group member is ignored in the document. However, the entire project is open-source and available on [GitHub](https://github.com/SimoneMarzeddu/Judy_Bot_T3).
<br>
<br>

## Relationship with the Course
The design phase of the agent exploited what is presented in AIMA Chapter 2 (second lesson of the course) to properly define the task environment and the agent performance measure. The technical core of the entire agent is certainly the implementation of the `A* Algorithm` supported by the `Octile Distance Heuristic`, developed according to what was learned from the fourth lesson of the course (AIMA Chapter 3).
<br>
<br>

## Environment and Software Dependencies
All Judy_Bot_T3 software dependencies are limited to the [NLE](https://github.com/facebookresearch/nle) installation.

The version of [NLE](https://github.com/facebookresearch/nle) used in the programming phase is 0.8.1.
Other software dependencies are closely related to the requirements for installing [NLE](https://github.com/facebookresearch/nle).
<br>
<br>



## How to run
To run the agent, simply use the shell command `python -m main` to start the code flow.

Thanks to the `config.json` configuration file it is possible to determine some aspects of the behavior of the software and the agent:

* The `task_prio_list` key allows you to define a list of tasks in order of priority (with their symbolic names).
The agent will consider the order given as a trace to administer his own logic, reserving the right to dynamically modify the established priorities in relation to the different game situations.
Currently, the agent has the ability to plan and execute 14 different tasks, which find their implementation in specific Python classes (see next section).
Below is a list of the symbolic names of the tasks accompanied by a brief description of their behavior. This specific order corresponds to the strategy that led Judy_Bot_T3 to the previously stated results:

  * `pray` -> Where the agent's prayer is planned, considering the requirements for safe prayer and the agent's needs,
  
  * `engrave_elbereth` -> Which allows the agent to engrave Elbereth on the ground, thus defending himself from some malevolent creatures,
  
  * `run_for_your_life` -> Which allows the agent to escape from unpleasant situations, fleeing from danger,
  
  * `take_a_break` -> Which allows the agent to rest and restore their vitality,
  
  * `close_monster_fight` -> 
Which allows the agent to fight, employing a strategy of avoiding passive monsters and not granting enemies bonus attacks,
  
  * `time_of_the_lunch` -> 
Which allows the agent to feed, avoiding eating dangerous food and checking for the presence of traps in suspicious corpses,
  
  * `greed_of_gold` -> Which allows the agent to reach and collect gold during his adventure,
  
  * `stairs_descent` -> Which allows the agent to descend into the dungeon according to a "slow descent" logic,
  
  * `stairs_ascent` -> Which allows the agent to go back up in the dungeon according to a "slow descent" logic,
  
  * `reach_closest_explorable` -> Which allows the agent to reach and interact with points of interest for exploration, such as doors and corridors,
  
  * `reach_horizon` -> Which allows the agent to reach the frontier of exploration, expanding their knowledge of the dungeon,
  
  * `explore_unseen` -> Which allows the agent to reach tiles they have never walked on,
  
  * `search_hidden_room` -> Which allows the agent to locate secret passages in dungeon rooms,
  
  * `search_hidden_corridor` -> Which allows the agent to locate secret passages in dungeon corridors.

    
* The `fast_mode` key determines how the agent will run. When the configured value is `on`, Judy_Bot_T3 will play NetHack without the terminal showing the game interface, saving computational resources for the massive execution of several games, printing a simple agent performance report.
When the configured value is `off`, the games played by Judy_Bot_T3 will be viewable through the typical game interface.


* The `attempts` key determines the number of games the agent will play.
<br>


## Code structure
The entire agent's logic is based on the modules that implement its tasks. Therefore, in order to expand the capabilities of the agent, it is necessary to extend the `Task` class or one of the other classes below it in the hierarchy, implementing the `planning()` and `execution()` methods, allowing the bot to integrate the task within its logic.

The following is a brief description of the main structural components of Judy_Bot_T3:

* `config.json` is the previously discussed configuration file,
* `main.py` is the startup component of the agent. Its code allows the parsing of the configuration file and the setting in motion of the whole logic of Judy_Bot-T3,
* `core.py` is the central component in interacting with the NLE framework and the underlying NetHack game,
* `archetype_modules.py` encompasses the three archetype classes for task definition: `Task` (the most general model), `ReachTask` (specialized in tasks that require to reach a specific glyph without too many frills) and `HiddenTask` (specialized in finding hidden areas),
* `reach_modules.py` includes classes that define tasks related to the `ReachTask` archetype,
* `secret_passage_modules.py` includes classes that define tasks related to the `HiddenTask` archetype,
* `general_modules.py` includes classes that define tasks related to the `Task` archetype, These are generic tasks and therefore not currently attributable to a more specific archetype.

## Example Configuration: Demo Mode

The following code shows Judy_Bot_T3 execution with the `fast-mode` configuration option set to `off` according to the example configuration file `config_oneshot` which provides the execution of only one game in `demo-mode`, activating the visualisation and rendering of the game interface so that the agent's activity is understandable to the human user. The console output shows the passing of game turns, deleting and rewriting the various screens to generate a frame-by-frame rendering of the game.

In [ ]:
from src.main import *

dungeon_walker, game, logic, task_map, attempts = start_bot("config/config_oneshot.json")

main_logic(dungeon_walker, game, logic, task_map, attempts)

## Example Configuration: Fast Mode

The following code snippet shows how Judy_Bot_T3 can be started with the `fast-mode` option set to `on` according to the example configuration file `config_fast100` which provides the execution of 100 games in the former mode. The console output summarises the scores results of the games played by the bot, without showing the specific progress of each game (no rendering of rounds, no display of details such as tasks performed or paths found), devoting the computational resources exclusively to the massive execution of the bot.

After each game's end, the console is updated to summarize the agent current `Mean` and `Median` scores, so to focus the user's attention on the core parameters of evaluation for the [NetHack Challenge at NeurIPS 2021](https://www.aicrowd.com/challenges/neurips-2021-the-nethack-challenge)

In [3]:
from src.main import *

dungeon_walker, game, logic, task_map, attempts = start_bot("config/config_fast100.json")

main_logic(dungeon_walker, game, logic, task_map, attempts)

// Mean :  0.0 // Median:  0  // Games:  0        (ಠ_ಠ)                
[]
